In [1]:
import json
from pprint import pprint
from odapui import yaml_reader
import pickle
from ruamel.yaml import YAML
from ruamel.yaml.comments import CommentedMap as ordereddict
from ruamel.yaml.scalarstring import SingleQuotedScalarString as sq
import copy
import sys
import odapui

In [3]:
rd = yaml_reader.Reader.get_instance()
yaml = YAML()
yaml.preserve_quotes = True
yaml.width = 40096
yaml.indent(mapping=4, sequence=6, offset=4)


initialising instance
start refreshing all yamls...2020-09-20T23:37:59.572788+10:00
adding element ZZ_FEED_ID2; [['FEED_NAME'], ['DB_NAME']]
adding element ZZ_FEED_ATTRIBUTE_ID_FLAT; [['FEED_ID', 'SOURCE_SYSTEM'], ['FEED_ID', 'FEED_NAME'], ['ATTRIBUTE_NAME']]
adding element ZZ_DATA_OBJECT_ATTRIBUTE_ID_FLAT; [['DATA_OBJECT_ID', 'DATA_OBJECT_NAME'], ['DATA_OBJECT_ID', 'TGT_DB_NAME'], ['ATTRIBUTE_NAME']]
adding element ZZ_FEED_ID; [['FEED_ATTRIBUTE_ID', 'FEED_NAME'], ['FEED_ATTRIBUTE_ID', 'SOURCE_SYSTEM']]
adding element ZZ_DATA_OBJECT_ID; [['DATA_OBJECT_ATTRIBUTE_ID', 'DATA_OBJECT_NAME'], ['DATA_OBJECT_ATTRIBUTE_ID', 'TGT_DB_NAME']]
finished refreshing all yamls...2020-09-20T23:38:39.825681+10:00


In [5]:
s = rd.feed_attr_data_object_attr.entries[0]

In [6]:
s

ordereddict([('FEED_ATTRIBUTE_ID', ordereddict([('ATTRIBUTE_NAME', 'browser'), ('FEED_NAME', 'browser'), ('SOURCE_SYSTEM', 'aa')])), ('DATA_OBJECT_ATTRIBUTE_ID', ordereddict([('ATTRIBUTE_NAME', 'browser'), ('DATA_OBJECT_NAME', 'browser'), ('TGT_DB_NAME', 'cds_aa')])), ('ZZ_FEED_ATTR_DATA_OBJECT_ATTR_ID', {'FEED_ATTRIBUTE_ID': ordereddict([('ATTRIBUTE_NAME', 'browser'), ('FEED_NAME', 'browser'), ('SOURCE_SYSTEM', 'aa')]), 'DATA_OBJECT_ATTRIBUTE_ID': ordereddict([('ATTRIBUTE_NAME', 'browser'), ('DATA_OBJECT_NAME', 'browser'), ('TGT_DB_NAME', 'cds_aa')])}), ('ZZ_FEED_ID', {'FEED_NAME': 'browser', 'SOURCE_SYSTEM': 'aa'}), ('ZZ_DATA_OBJECT_ID', {'DATA_OBJECT_NAME': 'browser', 'TGT_DB_NAME': 'cds_aa'})])

In [7]:
yaml.dump(s, sys.stdout)

FEED_ATTRIBUTE_ID: &id001 {'ATTRIBUTE_NAME': 'browser', 'FEED_NAME': 'browser', 'SOURCE_SYSTEM': 'aa'}      ### foreign key need to specify condition here
DATA_OBJECT_ATTRIBUTE_ID: &id002 {'ATTRIBUTE_NAME': 'browser', 'DATA_OBJECT_NAME': 'browser', 'TGT_DB_NAME': 'cds_aa'}      ### foreign key need to specify condition here
ZZ_FEED_ATTR_DATA_OBJECT_ATTR_ID:
    FEED_ATTRIBUTE_ID: *id001
    DATA_OBJECT_ATTRIBUTE_ID: *id002
ZZ_FEED_ID:
    FEED_NAME: 'browser'
    SOURCE_SYSTEM: 'aa'
ZZ_DATA_OBJECT_ID:
    DATA_OBJECT_NAME: 'browser'
    TGT_DB_NAME: 'cds_aa'


In [14]:
def single_quote(e: any, int_fields: list) -> any:
    # max 2 layers
    if isinstance(e, str):
        return sq(e)
    res = ordereddict()
    for k1, v1 in e.items():
        if isinstance(v1, dict):
            # break up v1
            temp1 = ordereddict()
            for k2, v2 in v1.items():
                if k2 in int_fields:
                    temp1[sq(k2)] = v2
                else:
                    temp1[sq(k2)] = sq(v2)
            temp1.fa.set_flow_style()  # one line
            if isinstance(v1, ordereddict):
                temp1.ca.comment = v1.ca.comment
            res[k1] = temp1
#             res[k1] = v1
        else:
            if k1 in int_fields:
                res[k1] = v1
            else:
                res[k1] = sq(v1)
    res.ca.comment = e.ca.comment  # preserve comments
    return res


In [15]:
s2 = single_quote(s, [])

In [16]:
yaml.dump(s2, sys.stdout)

FEED_ATTRIBUTE_ID: {'ATTRIBUTE_NAME': 'browser', 'FEED_NAME': 'browser', 'SOURCE_SYSTEM': 'aa'}             ### foreign key need to specify condition here
DATA_OBJECT_ATTRIBUTE_ID: {'ATTRIBUTE_NAME': 'browser', 'DATA_OBJECT_NAME': 'browser', 'TGT_DB_NAME': 'cds_aa'}             ### foreign key need to specify condition here
ZZ_FEED_ATTR_DATA_OBJECT_ATTR_ID: {'FEED_ATTRIBUTE_ID': 'ordereddict([(''ATTRIBUTE_NAME'', ''browser''), (''FEED_NAME'', ''browser''), (''SOURCE_SYSTEM'', ''aa'')])', 'DATA_OBJECT_ATTRIBUTE_ID': 'ordereddict([(''ATTRIBUTE_NAME'', ''browser''), (''DATA_OBJECT_NAME'', ''browser''), (''TGT_DB_NAME'', ''cds_aa'')])'}
ZZ_FEED_ID: {'FEED_NAME': 'browser', 'SOURCE_SYSTEM': 'aa'}
ZZ_DATA_OBJECT_ID: {'DATA_OBJECT_NAME': 'browser', 'TGT_DB_NAME': 'cds_aa'}


In [2]:
yaml = YAML()
yaml.preserve_quotes = True
yaml.width = 40096
yaml.indent(mapping=4, sequence=6, offset=4)


In [3]:
dd = {
    "A": "abc",
    "B": 1,
    "C": {
        "c1": "c111",
        "c2": 222
    }
}

In [4]:
def single_quote(e: any) -> any:
    # max 3 layers
    if isinstance(e, str):
        return sq(e)
    res = ordereddict()
    for k1, v1 in e.items():
        if isinstance(v1, dict):
            # break up v1
            temp1 = ordereddict()
            for k2, v2 in v1.items():
                temp1[k2] = sq(v2)
            res[k1] = copy.deepcopy(temp1)
        else:
            res[k1] = sq(v1)
    return res

In [9]:
res = single_quote(dd)

In [13]:
res.fa.

<bound method Format.flow_style of <ruamel.yaml.comments.Format object at 0x7f15ffa95fd0>>

In [5]:
with open("test_output.yaml", 'w') as f:
    yaml.dump(single_quote(dd), f)

In [6]:
! code test_output.yaml

In [8]:
res

NameError: name 'res' is not defined